In [1]:
from scipy.optimize import linprog

# a) 
# Digitos GRR
X = 6
Y = 4
Z = 1

PC = 2100 + (Z * 10) # Preco de venda de C por tonelada em reais
# Funcao objetivo
Fobj = (- PC + 2000, - PC + 1500) # Custos e preco de venda
# Inequacoes
# lado direito
LD = [[-0.04, 0.07], # inequacao 1 Carbono -4% * x + 7% * y <= 0 
      [-0.005, 0.005], # inequacao 2 Enxofre -0.5% * x + 0.5% * y <= 0
      [-0.03, Y * 1e-2], # inequacao 3 cinzas -3% * x + Y% * y <= 0
      [1, 1]] # inequacao 4 demanda - x - y <= - 20 - X  
# lado esquerdo
LE = [0, # inequacao 1 Carbono -4% * x + 7% * y <= 0 
      0, # inequacao 2 Enxofre -0.5% * x + 0.5% * y <= 0 
      0, # inequacao 3 cinzas -3% * x + Y% * y <= 0
      20 + X] # inequacao 4 demanda  - x - y <= - 20 - X
# Optimizacao por Simplex ++ 
opt = linprog(c = Fobj, A_ub = LD, b_ub = LE)
print(f"A:{opt.x[0]:.5f} B:{opt.x[1]:.5f}, Lucro: {opt.fun:.5f}")
opt = linprog(c = Fobj, A_ub = LD, b_ub = LE, integrality = 1)
print(f"A:{opt.x[0]} B:{opt.x[1]}, Lucro: {opt.fun}")

A:16.54545 B:9.45455, Lucro: -7587.27273
A:17.0 B:9.0, Lucro: -7360.0


In [2]:
import pulp
# b)
# problema
lucro = pulp.LpProblem("lucro", pulp.LpMinimize)
# Variaveis
C1 = pulp.LpVariable("C1", 0, 3000, pulp.LpInteger)
C2 = pulp.LpVariable("C2", 0, 2000, pulp.LpInteger)
C3 = pulp.LpVariable("C3", 0, 4000, pulp.LpInteger)
C4 = pulp.LpVariable("C4", 0, 1000, pulp.LpInteger)
A = pulp.LpVariable("A", 0, cat = pulp.LpInteger)
B = pulp.LpVariable("B", 0, cat = pulp.LpInteger)
C = pulp.LpVariable("C", 0, cat = pulp.LpInteger)
# equacoes e inequacoes
lucro += 13 * C1 + 15.3 * C2 + 14.6 * C3 + 14.9 * C4 - (16.2 * A + 15.75 * B + 15.3 * C), "lucro total" # funcao objetivo
# restricoes
lucro += ( C1 - 15 * 1e-2 * A <= 0 , "Blend C1 em A")
lucro += ( - C2  + 40 * 1e-2 * A <= 0 , "Blend C2 em A")
lucro += ( C3 - 50 * 1e-2 * A <= 0 , "Blend C3 em A")
lucro += ( C1 - 10 * B <= 0 , "Blend C1 em B")
lucro += ( - C2 + 10 * 1e-2 * B <= 0 , "Blend C2 em B")
lucro += ( C1 - 20 * 1e-2 * C <= 0 , "Blend C1 em C")
lucro += (C1 + C2 + C3 + C4 - (A + B + C) == 0, "conservacao de massa")

lucro.writeLP("Questao4b")
lucro.solve()
print("Status:", pulp.LpStatus[lucro.status])
for v in lucro.variables():
    print(v.name, "=", v.varValue)
print("lucro total = ", - pulp.value(lucro.objective))

Status: Optimal
A = 5000.0
B = 16.0
C = 605.0
C1 = 121.0
C2 = 2000.0
C3 = 2500.0
C4 = 1000.0
lucro total =  6935.5
